In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn import model_selection




import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("E:/research/Spacematch/formal research/feedback_89.csv") #import the data-points
# df.count()

In [3]:
y = df.sensations_temperature
X = df.drop(["user_id","place_id", "created_at","sensations_temperature","preferences_temperature"], axis = 1)  #Get X and y
# print(X.count())
rt_data = pd.read_csv("E:/research/Spacematch/formal research/real_time_room_data.csv")
rt_data = rt_data.drop(["room_id"], axis = 1)
X = X.append(rt_data)

In [4]:
def createNewTime(row):
    if row['Time'] <= 12:
        return 1
    else:
        return 2
    
X['New Time'] = X.apply(createNewTime, axis=1)
X = X.drop(['Time'], axis = 1)

rt_data = X.iloc[-4:]
X = X.iloc[:74]


In [5]:
X.head(20)

,clothing_level,indoor_temp,indoor_humidity,outdoor_temp,outdoor_humidity,New Time
0,1,76.86,39.82,64.94,84.273097,1
1,1,76.95,39.81,64.94,84.273097,1
2,2,69.39,48.94,64.94,84.273097,1
3,2,69.48,48.90,64.94,84.273097,1
4,2,75.27,40.21,64.94,84.273097,1
5,2,75.43,40.13,64.94,84.273097,1
6,2,73.19,42.66,69.08,70.285014,1
7,2,73.23,42.64,69.08,70.285014,1
8,2,73.23,42.62,69.08,70.285014,1
9,1,78.22,38.48,82.04,50.870173,2


In [6]:
param_test = {"C"               :0.1 ** np.linspace(0,5,10),
              "solver"          :['newton-cg', 'lbfgs', 'saga', 'sag']
              } # Optimize the parameters
cv_param = []
C_param = []
solver_param = []
accuracy_param = []
loo_accuracy_param = []

room_1_neutral = []
room_2_neutral = []
room_3_neutral = []
room_4_neutral = []
for i in range (2,9,1):
    gsearch = GridSearchCV(estimator = LogisticRegression(
        multi_class = 'multinomial', max_iter = 10000, random_state = 10), param_grid = param_test, cv = i)
    gsearch.fit(X, y)
    cv_param.append(i)
    solver_p    = gsearch.best_params_["solver"]
    C_p         = gsearch.best_params_["C"]
    C_param.append(C_p)
    solver_param.append(solver_p)
    accuracy_param.append(gsearch.best_score_)
       
    loo = LeaveOneOut() # test the accuracy with leaveoneout
    y_true = []
    y_pred = []
    for train_index, test_index in loo.split(X): # Split in X
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y.values[train_index], y.values[test_index]
        lr = LogisticRegression(multi_class = 'multinomial', max_iter = 10000, random_state = 10,
                                solver = solver_p, C = C_p,)
        lr = lr.fit(X_train, y_train)
        predict_y = lr.predict(X_test)
        y_true.append(y_test[0])
        y_pred.append(predict_y[0])
    accuracy = accuracy_score(y_true, y_pred)
    loo_accuracy_param.append(accuracy)
    
    lr = LogisticRegression(multi_class = 'multinomial', max_iter = 10000, random_state = 10
                            , solver = solver_p, C = C_p)
    lr.fit(X,y)
    
    rt_prepro = lr.predict_proba(rt_data)
    room_1_neutral.append(rt_prepro[0][1])
    room_2_neutral.append(rt_prepro[1][1])
    room_3_neutral.append(rt_prepro[2][1])
    room_4_neutral.append(rt_prepro[3][1])

In [7]:
result = pd.DataFrame(list(zip(cv_param, C_param, solver_param, accuracy_param, loo_accuracy_param, 
                               room_1_neutral, room_2_neutral, room_3_neutral, room_4_neutral)),
                      columns = ['cv', 'C', 'solver', 'accuracy', 'loo_accuracy', 
                                 'room_1', 'room_2', 'room_3', 'room_4'])

In [8]:
result.to_csv('E:/research/Spacematch/formal research/feedback_89_result_lr.csv')

In [9]:
# param_test = {"C"               :np.logspace(1,10,1),
#               "solver"          :['newton-cg', 'lbfgs', 'saga', 'sag']
#               } # Optimize the parameters
# gsearch = GridSearchCV(estimator = LogisticRegression(multi_class = 'multinomial', max_iter = 5000, random_state = 10),
#             param_grid = param_test, cv = 2)
# gsearch.fit(X, y)

In [10]:
# solver_p    = gsearch.best_params_["solver"]
# C_p         = gsearch.best_params_["C"]
# para = (solver_p, C_p, gsearch.best_score_)
# print(para)

In [11]:
# loo = LeaveOneOut() # test the accuracy with leaveoneout
# y_true = []
# y_pred = []
# for train_index, test_index in loo.split(X): # Split in X
#     X_train, X_test = X.values[train_index], X.values[test_index]
#     y_train, y_test = y.values[train_index], y.values[test_index]
#     lr = LogisticRegression(multi_class = 'multinomial', max_iter = 5000, random_state = 10
#                             , solver = solver_p, C = C_p)
#     lr = lr.fit(X_train, y_train) # train model
#     predict_y = lr.predict(X_test)
#     y_true.append(y_test[0])
#     y_pred.append(predict_y[0])
# print(y_true)
# print(y_pred)
# accuracy = accuracy_score(y_true, y_pred)
# print("accuracy is:", accuracy)

In [12]:
# # import the realtime room data
# rt_data = pd.read_csv("E:/research/Spacematch/formal research/real_time_room_data.csv")
# rt_data = rt_data.drop(["room_id"], axis = 1)
# rt_data.head()

In [13]:
# lr = LogisticRegression(multi_class = 'multinomial', max_iter = 5000, random_state = 10, solver = solver_p, C = C_p)
# lr.fit(X,y)
# lr.get_params()

In [14]:
# print(lr.predict(rt_data))
# rt_prepro = lr.predict_proba(rt_data)
# print("probability : \n", rt_prepro)